# ............................................................................................................................................

# Welcome to the RAAIM

Use shift + enter to run a code block or move to the next text block.

Run the next block of code to intialize the program.

* If you run into a "C:\Users\labuser\AppData\Local\Enthought\Canopy\User\lib\site-packages\pandas\io\excel.py:626 error", simply ignore it! The program will still be able to run.

In [1]:
import numpy as np
import scipy
from scipy import spatial
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import time as t
from PIL import Image
import sys
import os, errno
import matplotlib.patches as patches
from pandas.tools.plotting import autocorrelation_plot

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise

def load_RAAIM(folder, original):
    data = pd.read_csv(r'%s/ROI normalized.txt' %(folder), index_col= 'time(s)', sep='\t')
    print "Loaded 'ROI normalized.txt'"
    
    roi_param = pd.read_csv(r'%s/Parameter List_edit.txt' %(folder), index_col='ROI', sep='\t')
    print "Loaded 'Parameter List_edit.txt'"
    
    im = Image.open(r'%s/rgb.png' %(folder)) #MUST BE RGB
    print "Loaded 'rgb.png'"
    
    del data['Unnamed: 0'] #lc_pro outputs a weird blank column named this everytime. I don't know why, but it does. this line deletes it safely.
    roi_loc, roi_x, roi_y, data = lcpro_param_parse(roi_param, data , original = original)
    print "Configured Data"
    
    events_x, events_y = get_events(data = data, roi_param = roi_param)
    print "LCPro events extracted"
    
    path = folder +"/plots"
    mkdir_p(path)
    print "Made plots folder"
    
    return data, roi_param, im, roi_loc, roi_x, roi_y, events_x, events_y
    
def lcpro_param_parse(roi_param, data , original):
    '''
    This function takes the Dataframe created by opening the 'Parameter List.txt' from LC_Pro. 
    '''
    roi_loc = roi_param[['X', 'Y']]
    roi_loc.drop_duplicates(inplace= True)
    roi_x = roi_loc['X'].tolist()
    roi_y = roi_loc['Y'].tolist()
    new_index = []
    for i in np.arange(len(roi_loc.index)):
        foo = int(roi_loc.index[i])
        new_index.append('Roi'+str(foo))
    roi_loc = DataFrame({'x':roi_x, 'y':roi_y}, index= new_index)
    
    if len(data.columns) != len(new_index) and original == True:
        sys.exit("The number of ROIs in the data file is not equal to the number of ROIs in the parameter file. That doesn't seem right, so I quit the function for you. Make sure you are loading the correct files, please.")
    
    if original == False:
        data = data[roi_loc.index]
    
    truth = (data.columns == roi_loc.index).tolist()
    
    if truth.count(True) != len(data.columns):
        sys.exit("The names on data and roi_loc are not identical. This will surely break everything later, so I shut down the program. Try loading these files again.")
    
    return roi_loc, roi_x, roi_y, data

def get_events(data, roi_param):
    '''
    extract the events from the roi_parameter list. It returns them as a pair of dictionaries.
    '''
    new_index = []
    for i in np.arange(len(roi_param.index)):
        new_index.append('Roi'+str(roi_param.index[i]))
    roi_events = DataFrame(index= new_index)
    roi_events_time = roi_param['Time(s)'].tolist()
    roi_events_amp = roi_param['Amp(F/F0)'].tolist()
    roi_events['Time'] = roi_events_time
    roi_events['Amp'] = roi_events_amp
    
    events_x = {}
    events_y = {}
    for label in data.columns:
        events_x[label] = []
        events_y[label] = []

    for i in np.arange(len(roi_events.index)):
        key = roi_events.index[i]
        events_x[key].append(roi_events.iloc[i,0])
        events_y[key].append(roi_events.iloc[i,1])
        
    return events_x, events_y

def cross_correlation(data, r):
    '''
    This function calculated the cross correlation between each ROI pair and returns a dataframe with NxN+1  values.
    it also includes a column that contains the number of friends each ROI has.
    lastly, it creates a dictionary where the keys are the roi names and names of each roi's friends are stored as a list. by defalut, each roi is at least friends with itself.
    this file can be saved out as a CSV later.
    '''
    
    data_corr = data.corr()
    ls = (data_corr[data_corr>r].count())
    data_corr['Number of Friends'] = ls
    cols = data_corr.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    data_corr = data_corr[cols]
    
    friend_names = {}
    temp_cor = data.corr()
    correlation_summary = DataFrame()
    
    for label, column in temp_cor.iteritems():
        friend_names[label] = column[column>r].index.tolist()
        friends = column[column>r]
        summary = friends.describe()
        correlation_summary[label] = np.round(summary, 3)

    return data_corr, friend_names, correlation_summary

def distance_matrix(roi_x, roi_y , roi_loc , d):
    '''
    This function calculates the euclidean distance between each ROI pair. It returns a dataframe with these distances.
    it also includes a column that contains the number of neighbors each ROI has.
    the files can be saved out as a CSV later.
    It also returns the list of the (x,y) tuple for each roi.
    '''
    loc_tup = []
    for i in np.arange(len(roi_x)):
        loc_tup.append((roi_x[i], roi_y[i]))
    
    roi_dist = DataFrame(data = (scipy.spatial.distance.cdist(loc_tup, loc_tup, 'euclidean')), index = roi_loc.index, columns= roi_loc.index)
    roi_dist_f = roi_dist
    
    neighbor_names = {}
    distance_summary = DataFrame()
    
    for label, column in roi_dist.iteritems():
        neighbor_names[label] = column[column<=d].index.tolist()
        neighbors = column[column<=d]
        summary = neighbors.describe()
        distance_summary[label] = np.round(summary, 3)

    ls = (roi_dist[roi_dist<=d].count())
    roi_dist_f['Number of Neighbors'] = ls
    cols = roi_dist_f.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    roi_dist_f = roi_dist_f[cols]
    
    return roi_dist_f, loc_tup, neighbor_names, distance_summary

def friend_dist(data_corr, roi_dist, friend_names, r, d):
    '''
    calculates distance information about each roi's friends.
    returns a dataframe with Distance Sum, number of friends, and mean distance. mean dist = distance sum/number of fiends
    '''
    roi_friend_dist = DataFrame(index = data_corr.index)
    #roi_friend_dist['Distance sum'] = (roi_dist[data_corr>r].sum())
    roi_friend_dist['Number of Friends'] = (data_corr[data_corr>r].count())
    roi_friend_dist['Mean Dist'] = (roi_dist[data_corr>r].mean())
    roi_friend_dist['Std Dist'] = (roi_dist[data_corr>r].std())
    
    perc_friends_are_neighbors = Series(index = roi_dist.index) #empty series
    for key in friend_names:
        temp_series = roi_dist.loc[key, friend_names[key]]
        count = temp_series[temp_series<=d].count()
        perc_friends_are_neighbors[key] = count
    roi_friend_dist['Number of Friends that are Neighbors'] = perc_friends_are_neighbors
    roi_friend_dist['Percentage of Friends that are Neighbors'] = (roi_friend_dist['Number of Friends that are Neighbors']/roi_friend_dist['Number of Friends'])*100
    
    return roi_friend_dist

def neighbor_corr(roi_dist, data_corr, neighbor_names, r, d):
    '''
    Calculates correlation information about each roi's neighbors.
    returns dataframe with corr of neighbors, corr of neighbors that are friends, percent of neighbors that are friends
    '''
    
    roi_neighbor_corr = DataFrame(index = roi_dist.index)
    roi_neighbor_corr['Number of Neighbors'] = (roi_dist[roi_dist<=d].count())
    roi_neighbor_corr['Mean Correlation'] = (data_corr[roi_dist<=d].mean())
    roi_neighbor_corr['Std Correlation'] = (data_corr[roi_dist<=d].std())
    
    perc_neighbors_are_friends = Series(index = roi_dist.index)
    for key in neighbor_names:
        temp_series = data_corr.loc[key, neighbor_names[key]]
        count = temp_series[temp_series>r].count()
        perc_neighbors_are_friends[key] = count
    roi_neighbor_corr['Number of Neighbors that are Friends'] = perc_neighbors_are_friends
    roi_neighbor_corr['Percentage of Neighbors that are Friends'] = (roi_neighbor_corr['Number of Neighbors that are Friends']/roi_neighbor_corr['Number of Neighbors'])*100
    
    return roi_neighbor_corr

def onpick(event):
    '''
    allows us to click on an roi and print in the notebook it's name.
    '''
    ind = event.ind
    ind = ind[0]
    print('Roi selected =', roi_loc.index[ind])
    
def all_roi_2d(roi_loc , im , s=6):
    '''
    makes a scatter plot of each roi overlayed on an uploaded image. when you click on an roi, it prints the name of the roi
    for some reason, it takes two clicks to start printing. Not sure why
    '''
    fig, ax = plt.subplots()
    col = ax.scatter(roi_loc.x, roi_loc.y, s=s, edgecolor = 'k', linewidth ='1',color = 'y', marker ="o", alpha = 0.75, picker = True)
    fig.canvas.mpl_connect('pick_event', onpick)

    width, height = im.size
    plt.xlim(xmin=0, xmax=width)
    plt.ylim(ymin = 0, ymax = height)
    plt.title('All LC_Pro ROIs')
    plt.imshow(im)
    plt.show()
    
def friend_roi_2d(data_corr , friend_names, roi_loc, r, d, roi, just_friends, s = 6):
    temp_series_list = data_corr[roi].tolist()
    temp_series = data_corr[roi]
    temp_ls = temp_series[friend_names[roi]]

    fig, ax = plt.subplots()
    if just_friends == False:
        plt.scatter(roi_loc.x, roi_loc.y, s = s, label = 'other objects', edgecolor = 'k', linewidth ='1',c= temp_series_list,cmap=plt.cm.spectral , marker ="o", alpha = 1)
        plt.colorbar()
        plt.clim(-1,1)

        
    if just_friends == True:    
        plt.scatter(roi_loc.x, roi_loc.y, s = s, label = 'other objects', edgecolor = 'k', linewidth ='1',color= 'w' , marker ="o", alpha = 0.25)
        plt.scatter(roi_loc.x[friend_names[roi]], roi_loc.y[friend_names[roi]], label = 'Friends', s=s, edgecolor = 'k', linewidth ='1',c =temp_ls, cmap=plt.cm.spectral , marker ="o", alpha = 1)
        plt.colorbar()
        plt.clim(r,1)
        
    plt.scatter(roi_loc.x[roi], roi_loc.y[roi], label = roi, s=(s+25), edgecolor = 'k', marker ="o", alpha = 1, color = 'w')
    neighborhood = patches.Circle(xy = (roi_loc.x[roi], roi_loc.y[roi]), radius=d, fill = False, color= 'w', lw = 1, ls = 'solid')
    ax.add_artist(neighborhood)
    
    width, height = im.size
    plt.xlim(xmin=0, xmax=width)
    plt.ylim(ymin = 0, ymax = height)
    plt.title(roi)
    plt.imshow(im)
    #plt.legend(loc = 'center left')

    plt.show()
    
def all_roi_line_pdfs(dataframe, num_rois, ymin, ymax, events_x, events_y, folder):
    
    for r in np.arange(num_rois):
        
        plt.figure()
        plt.xlabel('Time (s)')
        plt.ylabel('Intensity')
        plt.title(dataframe.columns[r])
        plt.ylim(ymin = ymin, ymax = ymax)
        plt.xlim(xmin= dataframe.index[0], xmax = dataframe.index[-1])
        plt.plot(dataframe.index, dataframe.ix[:,r], color = [np.random.random(), np.random.random(), np.random.random()])
        plt.plot(events_x[dataframe.columns[r]], events_y[dataframe.columns[r]],  marker = "^", color="y", linestyle= "None")
        
        plt.savefig(r'%s/plots/%s.pdf' %(folder,dataframe.columns[r]))
        plt.close()
        
def roi_line_plot(roi, dataframe, events_x, events_y, ymin, ymax):
    
    plt.figure()
    plt.xlabel('Time (s)')
    plt.ylabel('Intensity')
    plt.title(roi)
    plt.ylim(ymin = ymin, ymax = ymax)
    plt.xlim(xmin= dataframe.index[0], xmax = dataframe.index[-1])
    plt.plot(dataframe.index, dataframe.ix[:,roi], color = 'k')
    plt.plot(events_x[roi], events_y[roi],  marker = "^", color="y", linestyle= "None")

    plt.show()
    
    
def friend_roi_line(data, friend_names, roi):
    plt.figure()
    plt.plot(data.index, data[friend_names[roi]], color = '#FF6600')
    plt.plot(data.index, data[roi], color = 'k',)
    plt.ylim(ymin = 0, ymax = max(data.max()))
    plt.title(roi)

    plt.show()
    
def friend_roi_line_average(data, friend_names, roi):
    roi_average = data[friend_names[roi]].mean(1)
    roi_std = data[friend_names[roi]].std(1)
    print "the average of the std of each point is", round(np.mean(roi_std),4), "for", roi, "and its friends."

    plt.figure()
    plt.plot(data[friend_names[roi]].mean(1))
    plt.ylim(ymin = 0, ymax = max(data.max()))
    plt.title('Average of '+roi+' and its friends')
    plt.show()

def boxandwhiskers(roi, flavor,data_corr, roi_dist, r, d):
    '''
    Returns the box and whiskers plot of a named ROI's friends or neighbors.
    
    '''
    if flavor == 'corr':
        temp_series = data_corr[roi]
        temp_series = temp_series[temp_series>r]
        plt.boxplot(temp_series)
        plt.ylim(ymin = r*0.85, ymax = 1.1)
        plt.xticks(np.arange(1),roi)
        plt.title(roi)
        plt.text(0.55,r,temp_series.describe())
        plt.ylabel('Correlation (r)')
        plt.show()
        
    
    elif flavor == 'dist':
        temp_series = roi_dist[roi]
        temp_series = temp_series[temp_series<=d]
        plt.boxplot(temp_series)
        plt.ylim(ymin = -3, ymax = d*1.25)
        plt.xticks(np.arange(1),roi)
        plt.title(roi)
        plt.text(0.55,d*0.5,temp_series.describe())
        plt.ylabel('Distance (pixels)')
        plt.show()
        
        
    else:
        print "ERROR: flavor must be set to either 'corr' or 'dist'."        
        
print "RAAIM Ready for Analysis"

ImportError: No module named numpy

# I. Begin User Input

This whole notebook is especially formated for LC_pro outputs. This means that the load functions and wrappers expect that certain columns are in exactly the right locations and labels are formatted a certain way. This author does not expect other types of files to be handled properly (as in: will probably fail). 

## ------------------------------- An Important note about saving!  -------------------------------

This notebook is brutal and will mercilessly **save over exsisting files of the same name without warning!**

You can manually change the name of the saved file one way: 

Change part of the text portion of the VARIBLE.to_csv(r'%s/**OBJECT**.csv' %(folder)). you can change **OBJECT** to whatever you want. go nuts. 

Remember, only you can keep all your files organized and named properly. I'm just here to help. Use a naming convention that is simple and easy to understand/remember. You can totally move ALL of these files to anywhere once you've run this analysis. It won't break stuff.

## ------------------------------------------------------------------------------------------------------

## A. Declare Your Folder

Change the following string below to the location of the folder where your data is. You will also save all of your data in this folder. Must be full path name and not end in a /.

**Example:**

"/Users/abigaildobyns/Desktop/RAAIM"

In [2]:
folder = "H:\Senior Research\Friends and neighbors\AN Cont\Animal1"

## B. Load

Use the following block below to load your data. There should be three files in this folder:

1. 'ROI normalized.txt'

2. 'Parameter List_edit.txt'

3. 'rbg.png'

There should be no headers (other than column names) in the text files. the text files can be either .txt or .csv. if you have problems loading them, make sure that the data is not corrupted, it is in the correct file type, the delimiter is correct, and the file path is correct. 

the image still can be just a random frame or the averaged or compressed stack. it must be convered to RGB and saved as .png.

###### if you get a *SettingWithCopyWarning*,  **Don't worry, that's normal**. I know that you are setting with a copy. It's on purpose

In [3]:
data, roi_param, im, roi_loc, roi_x, roi_y, events_x, events_y = load_RAAIM(folder=folder, original=False)

Loaded 'ROI normalized.txt'
Loaded 'Parameter List_edit.txt'
Loaded 'rgb.png'
Configured Data
LCPro events extracted
Made plots folder


C:\Users\labuser\AppData\Local\Enthought\Canopy\User\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [4]:
roi_loc

,x,y
Roi8,286,152
Roi38,283,197
Roi40,382,267
Roi41,152,180
Roi46,97,221
Roi67,402,272
Roi72,419,224
Roi77,384,320
Roi78,405,163
Roi85,147,216


###### Use the two cells below to check that the data and roi_loc dataframes look as expected.

In [5]:
data.head()

,Roi8,Roi38,Roi40,Roi41,Roi46,Roi67,Roi72,Roi77,Roi78,Roi85,...,Roi198,Roi199,Roi200,Roi202,Roi205,Roi208,Roi211,Roi212,Roi229,Roi232
time(s),,,,,,,,,,,,,,,,,,,,,
0.78125,0.874040,0.935501,1.036784,0.933514,0.941615,0.929188,0.928453,0.984584,0.935582,0.957572,...,0.959209,1.016769,0.990942,0.953677,1.014108,0.902092,1.018256,0.991692,0.956017,0.908226
1.56250,0.922546,0.927720,1.025545,0.925047,0.946316,0.929188,0.919237,0.969620,0.909053,0.963721,...,0.971720,1.025929,0.958386,0.974085,1.020840,0.943020,0.966019,1.009420,0.873850,0.927756
2.34375,0.904049,0.933760,1.064874,0.932240,0.957243,0.957901,0.961278,0.948195,0.950094,0.920644,...,0.976408,1.050185,0.995857,0.947758,1.047189,0.930049,0.960530,0.972846,0.897499,0.939467
3.12500,0.878145,0.932903,1.087352,0.931820,0.959206,0.914605,0.979709,1.000536,0.959631,0.909753,...,0.968067,1.034555,0.990332,0.968820,1.003258,0.966983,1.000117,0.992241,0.887434,0.974618
3.90625,0.929123,0.923837,1.085746,0.918693,0.925986,0.947868,0.931899,0.933726,0.937240,0.932473,...,0.945129,1.008143,1.001400,0.951718,1.032203,0.943020,0.978669,1.027149,0.897499,0.890374


In [6]:
roi_loc.head()

,x,y
Roi8,286,152
Roi38,283,197
Roi40,382,267
Roi41,152,180
Roi46,97,221


# II. Analysis

## A. Cross Correlation Matrix (friends)

Set your r value to your threshold for significant cross correlation. This value will be used is all later calculation. If you change it, you must rerun ALL cells following to update the change.

In [7]:
r = 0.8 #set r value, must be -1<= r <=1

In [8]:
data_corr, friend_names, correlation_summary = cross_correlation(data, r)
data_corr.head()

TypeError: round() takes at most 2 arguments (3 given)

### - Cross Correlation Matrix Summary

In [11]:
correlation_summary

NameError: name 'correlation_summary' is not defined

#### Like what you see? save the cross correlation file with the below code

In [10]:
data_corr.to_csv(r'%s/xcor.csv' %(folder))
correlation_summary.to_csv(r'%s/xcor_summary.csv'%(folder))

## B. Distance Matrix (neighbors)

Set your d value to your threshold for significant distance. This value will be used is all later calculation. If you change it, you must rerun ALL cells following to update the change.

In [11]:
d = 100 # set the maximum distance from an ROI that is the threshold for neighborhood

In [12]:
roi_dist, loc_tup, neighbor_names, distance_summary = distance_matrix(roi_x, roi_y , roi_loc , d)
roi_dist.head()

,Number of Neighbors,Roi3,Roi11,Roi18,Roi21,Roi23,Roi25,Roi32,Roi40,Roi42,...,Roi244,Roi245,Roi248,Roi250,Roi255,Roi257,Roi258,Roi260,Roi262,Roi265
Roi3,11,0.000000,306.341313,238.119718,275.465061,276.725496,344.013081,410.855206,307.865230,252.067451,...,260.340162,187.032083,72.062473,52.773099,362.215406,550.158159,210.011904,468.600043,479.292186,492.212353
Roi11,21,306.341313,0.000000,360.169405,385.167496,255.890602,341.013196,359.456534,19.697716,232.561820,...,214.462118,342.116939,342.230916,295.489425,323.241396,279.028672,125.483067,238.102919,252.071418,212.438226
Roi18,17,238.119718,360.169405,0.000000,37.576588,138.513537,141.449638,215.603803,348.313078,143.013985,...,442.922115,51.788030,308.038959,186.096749,176.581426,489.376133,237.191905,385.923568,390.632820,453.616578
Roi21,16,275.465061,385.167496,37.576588,0.000000,147.410990,126.063476,197.739728,372.120948,158.911925,...,478.012552,88.600226,345.050721,223.579963,164.805946,494.231727,264.847126,389.370004,392.787474,463.020518
Roi23,10,276.725496,255.890602,138.513537,147.410990,0.000000,90.210864,139.043159,239.908316,30.083218,...,397.061708,155.241747,347.070598,228.442553,90.426766,350.891721,154.201167,247.872951,253.179778,316.180328


### - Distance Matrix Summary

In [13]:
distance_summary

,Roi3,Roi11,Roi18,Roi21,Roi23,Roi25,Roi32,Roi40,Roi42,Roi46,...,Roi244,Roi245,Roi248,Roi250,Roi255,Roi257,Roi258,Roi260,Roi262,Roi265
count,11.000,21.000,17.000,16.000,10.000,10.000,8.000,19.000,9.000,8.000,...,1,17.000,3.000,12.000,10.000,8.000,11.000,6.000,5.000,9.000
mean,55.410,50.697,38.540,53.978,64.015,50.524,43.078,50.127,50.501,45.954,...,0,44.662,51.054,38.485,50.116,50.697,52.351,42.539,32.693,60.339
std,29.334,29.273,25.506,32.073,36.530,27.424,24.245,29.992,38.780,26.082,...,NaN,25.754,44.444,29.203,27.525,30.736,31.394,42.063,39.031,41.505
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,45.304,31.016,21.024,30.403,33.408,34.748,33.155,25.511,13.601,39.983,...,0,26.926,36.031,17.770,42.486,34.214,26.152,15.244,14.036,11.045
50%,61.008,47.202,34.785,54.300,85.506,47.826,51.231,44.721,55.902,50.810,...,0,42.107,72.062,32.903,55.938,47.728,64.031,25.442,18.000,82.079
75%,76.658,76.694,51.788,78.400,90.373,71.462,55.922,79.506,86.145,59.771,...,0,58.600,76.581,52.114,65.211,69.268,77.413,77.236,31.953,93.814
max,89.560,96.426,96.167,99.000,99.690,90.211,74.303,97.739,98.858,81.271,...,0,98.234,81.099,90.824,90.427,99.960,94.763,98.005,99.479,98.005


#### Like what you see? save the distance matrix file with the below code

In [14]:
roi_dist.to_csv(r'%s/dist_matrix.csv' %(folder))
distance_summary.to_csv(r'%s/dist_summary.csv' %(folder))

## C. Friends in terms of Neighbors

the following function returns a table of the following values:

**Distance Sum**: the sum of the distances from each roi to its friends

**Friend Number**: the number of friends that each roi has (not including itself)

**Rel Dist**: The cummulative distance / number of friends

In [15]:
friend_distances = friend_dist(data_corr, roi_dist, friend_names, r, d)
friend_distances.head()

,Number of Friends,Mean Dist,Std Dist,Number of Friends that are Neighbors,Percentage of Friends that are Neighbors
Roi3,4,33.472739,34.536468,4,100
Roi11,5,26.595634,25.191076,5,100
Roi18,8,38.099606,33.998655,8,100
Roi21,9,48.222222,36.179337,9,100
Roi23,5,37.761798,35.253486,5,100


### - Friends in terms of Neighbors Summary

In [16]:
friend_dist_sum = friend_distances.describe()
friend_dist_sum

,Number of Friends,Mean Dist,Std Dist,Number of Friends that are Neighbors,Percentage of Friends that are Neighbors
count,96.000000,96.000000,75.000000,96.000000,96.000000
mean,4.166667,25.259734,30.285566,3.958333,96.380208
std,2.614400,26.406310,31.062494,2.495962,10.955597
min,1.000000,0.000000,5.742172,1.000000,25.000000
25%,2.000000,6.762956,13.161736,2.000000,100.000000
50%,4.000000,18.509496,23.259407,4.000000,100.000000
75%,5.000000,37.793571,35.124975,5.000000,100.000000
max,9.000000,144.930993,226.972085,9.000000,100.000000


#### Like what you see? save the friend distances file with the below code

In [17]:
friend_distances.to_csv(r'%s/friend_distances.csv' %(folder))
friend_dist_sum.to_csv(r'%s/friend_distances_summary.csv' %(folder))

## D. Neighbors in terms of Friends

This section has yet to be built. Will talk with Sean Wilson about values that make the most sense. here. perhaps number of neightbors and average r value?

In [18]:
neighbor_correlation = neighbor_corr(roi_dist, data_corr, neighbor_names, r, d)
neighbor_correlation.head()

,Number of Neighbors,Mean Correlation,Std Correlation,Number of Neighbors that are Friends,Percentage of Neighbors that are Friends
Roi3,11,0.328093,0.490787,4,36.363636
Roi11,21,0.408476,0.366014,5,23.809524
Roi18,17,0.700697,0.256872,8,47.058824
Roi21,16,0.752055,0.249099,9,56.250000
Roi23,10,0.471808,0.520604,5,50.000000


### - Neighbors in terms of Friends Summary

In [19]:
neighbor_corr_sum = neighbor_correlation.describe()
neighbor_corr_sum

,Number of Neighbors,Mean Correlation,Std Correlation,Number of Neighbors that are Friends,Percentage of Neighbors that are Friends
count,96.000000,96.000000,95.000000,96.000000,96.000000
mean,13.020833,0.452703,0.375028,3.958333,32.657700
std,4.881715,0.192370,0.110036,2.495962,19.823026
min,1.000000,0.029215,0.089952,1.000000,4.761905
25%,9.000000,0.330093,0.268144,2.000000,16.761364
50%,13.000000,0.413700,0.380642,4.000000,31.666667
75%,17.000000,0.554661,0.446816,5.000000,45.855615
max,23.000000,1.000000,0.585422,9.000000,100.000000


#### Like what you see? save the neighbor correlation file with the below code

In [20]:
neighbor_correlation.to_csv(r'%s/neighbor_correlation.csv' %(folder))
neighbor_corr_sum.to_csv(r'%s/neighbor_correlation_summary.csv' %(folder))

# III. Resulting Plots

## A. 2d Plots

### - All Roi overlay with name calling

For some reason, you have to click on the first cell twice. but be assured, it is working. set the object size with s. default is s = 6

Save this plot from the pop up graph window.

In [21]:
all_roi_2d(roi_loc , im , s=100)

('Roi selected =', 'Roi221')
('Roi selected =', 'Roi141')


### - Select Roi and Friends plot

Call a 2d plot with each roi on it. Choose an roi by name (ex: 'Roi35') to see it's relationship to all other rois. The named roi will be in white. All other roi's will be colored by its r value. 

when just_friends = True, only friends who meet the r threshold will be shown and all others will be gray. when just_friends = False, all rois will be colored based on r value.

Save this plot from the pop up graph window.

In [22]:
friend_roi_2d(data_corr , friend_names, roi_loc, r, d, roi = 'Roi221', just_friends = True, s = 25)

## B. Line Plots

All line plots are scaled the same: 0 to the maximum value in the entire dataframe.

### - Line Plots for all ROIs

Use all_roi_line_pdfs to plot each ROI as its own pdf. Each plot is randomly assigned a color. You cannot disable this feature.

you can easily consolidate all of these into a single pdf using adobe or another preview program.

In [23]:
all_roi_line_pdfs(data, num_rois = len(data.columns), ymin = 0, ymax=max(data.max()), events_x = events_x, events_y = events_y, folder = folder)

### - Line Plot for only one ROI

Use roi_line_plot to call an ROI and get its plot. It includes the overlay of the LCPro events

In [27]:
roi_line_plot(roi = 'Roi221', dataframe = data, events_x = events_x, events_y=events_y, ymin = 0, ymax=max(data.max()))

KeyError: 'Roi37'

### - Stacked line plot of an ROI and its friends

Call an roi by name (ex: 'Roi35') to get a line plot of it and its friends. Save this plot from the pop up graph window.

In [24]:
friend_roi_line(data, friend_names, roi = 'Roi221')

### - Average line plot of an ROI and its friends

Call an roi by name (ex: 'Roi35') to get a line plot of the average of it and its friends. the average standard error for each point is printed below once the plot is closed. Save this plot from the pop up graph window.

In [31]:
friend_roi_line_average(data, friend_names, roi = 'Roi37')

the average of the std of each point is 0.0727 for Roi37 and its friends.


### - Autocorrelation Plot

In [32]:
roi = 'Roi37'

autocorrelation_plot(data[roi])
plt.title(roi)
plt.show()

## C. Box and Whister Plots

Call an ROI and see the distribution of either its friends' correlations or neighbors' distances.

Choose which metrc to use by changing flavor to either 'dist' or 'corr'

In [33]:
boxandwhiskers(roi = 'Roi37', flavor = 'corr', data_corr = data_corr, roi_dist = roi_dist, r = r, d = d)

# End

# ............................................................................................................................................

## [ Run the next cells of code if you want to randomize the dataset ]

In [34]:
import random

def shuffle(x):
    random.shuffle(x)
    return x

for label, column in data.iteritems():
    tempcol = data[label].tolist()
    randcol = shuffle(tempcol)  #shuffle the list of values in the column
    data[label] = randcol #write over the original list with randomized list

### Check that the dataset set is indeed randomized! Compare this table to the table under the header "(b) Load"

In [35]:
data.head()

,Roi16,Roi26,Roi27,Roi29,Roi32,Roi33,Roi35,Roi36,Roi37,Roi41,...,Roi53,Roi54,Roi61,Roi62,Roi66,Roi71,Roi72,Roi73,Roi74,Roi76
time(s),,,,,,,,,,,,,,,,,,,,,
0.78125,1.607642,1.080892,1.020448,1.053762,0.941500,0.957506,1.118078,1.061659,0.953561,1.395612,...,1.012658,1.026843,1.090384,1.210970,1.058431,0.978613,0.998858,0.961801,1.020476,0.945394
1.56250,1.553273,1.031149,0.998754,0.962334,0.969865,1.014386,0.939536,0.967438,0.957473,1.061042,...,0.994998,0.920157,1.178071,1.074218,0.942334,1.072712,0.899077,1.215734,1.029408,0.931584
2.34375,0.962505,1.078617,0.954115,1.015816,1.213816,1.231693,1.001482,1.303174,0.965297,1.008439,...,0.955814,0.915933,0.927909,1.066917,1.170222,1.206210,1.180306,1.038254,1.117227,1.145272
3.12500,0.959802,1.032058,0.991184,0.959647,0.937691,0.999093,0.977013,0.971098,0.910528,0.813503,...,0.925442,1.090510,1.389492,1.025967,0.992697,1.069824,0.969827,0.909245,0.981771,1.001213
3.90625,0.977164,1.042051,1.382365,1.078552,1.044406,0.904864,0.949430,0.974289,0.973606,0.951778,...,1.039493,1.063521,0.986382,1.108594,1.049821,1.093856,1.051493,0.954309,1.061396,0.952597


### Now, go to analysis and it will run the program with the randomized data. (SKIP I. User Input) 

# ............................................................................................................................................